# Using Amazon SageMaker with Public Datasets

__*Clustering Gene Variants into Geographic Populations*__


## Introduction

Amazon SageMaker allows you to bring powerful machine learning workflows to data that is already in the cloud.  In this example, we will do just that - combining Amazon SageMaker with data from the [1000 Genomes Project] which is hosted by AWS as a [public dataset].  Specifically, we will perform unsupervised learning using Amazon SageMaker's KMeans algorithm to see if we can predict the geographic population for a set of single nucleotide polymorphisms.

Single nucleotide polymorphisms or SNPs (pronounced "snips") are single base-pair changes to DNA.  DNA is a long chain molecule that is used to store the "source code" for all living organisms and is "read" as a sequence of four nucleotides: A, T, C, and G.  A single letter is called a "base".  SNPs occur when one of these bases in the sequence changes due to environmental causes or random replication errors during cell division in germ cells (eggs and sperm).  Sometimes these changes are harmless, and sometimes they can cause serious diseases.

Here we are going to cluster high frequency SNPs found on Chromosome 6

### Attribution
This notebook is based on work previously described by [Databricks using Spark][databricks blog]

[1000 Genomes Project]: https://aws.amazon.com/1000genomes/
[public dataset]: https://aws.amazon.com/public-datasets/
[databricks blog]: https://databricks.com/blog/2016/05/24/predicting-geographic-population-using-genome-variants-and-k-means.html

## Setup

> This notebook was created and tested on an `ml.m4.2xlarge` notebook instance

Let's start by:

1. Downloading the data we need from S3
1. Installing some utility packages for processing the data

### Data sources
We can get variant call data (which describes SNPs) from the publicly hosted 1000 Genomes dataset on AWS.  We are need the "\*.vcf" file corresponding to Chromosome 6 from the 20130502 release of the data.

In [ ]:
%%bash
aws s3 ls --human-readable s3://1000genomes/release/20130502/ | grep chr6

The data for Chromosome 6 is nearly 1GB in size.  For the purpose of this exercise and to be conservative of space (the scratch area for sagemaker notebooks only have about 5GB of space) we are going to use a sub-sample of the data.  To generate that, we can use `tabix` a bioinformatics command line utility found in the [htslib] set of tools.

[htslib]: https://github.com/samtools/htslib

A version of `tabix` compiled for Linux was created and published to S3.  We'll download that into our SageMaker environment and use it to sample the Chromosome 6 VCF file __*directly on S3*__ and create a data file we can use here for model training.  Here we've reduced the data to entries found between positions 1000000-1250000.

In [ ]:
%%bash

aws s3 cp s3://pwyming-utils/htslib/tabix .

chmod +x ./tabix

./tabix -h \
    s3://1000genomes/release/20130502/ALL.chr6.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
    6:1000000-1250000 > 6-sample.vcf

Now let's grab metadata - information about geographic locations of where sample sequences came from - to use as labels in our model training.

In [ ]:
%%bash

aws s3 cp s3://1000genomes/release/20130502/integrated_call_samples_v3.20130502.ALL.panel .

## Exploration

To make exploring and processing the data a little easier, we'll use the `scikit-allel` package.  While this package does not come included with the SageMaker environment, it is easy to install.

More information about it can be found at:
http://scikit-allel.readthedocs.io/en/latest/index.html

It has good utilities for reading VCF files
http://alimanfoo.github.io/2017/06/14/read-vcf.html

In [ ]:
!conda install -c conda-forge -y scikit-allel

Now we can easily read in our sampled VCF file for data exploration

In [ ]:
import allel

callset = allel.read_vcf('6-sample.vcf')

The above returns a Python dictionary with keys that represent parts of the data

In [ ]:
callset.keys()

How many variants and samples are in this data?

In [ ]:
print("Samples: {samples}, Variants: {variants}".format(
    variants=len(callset['calldata/GT']), 
    samples=len(callset['samples']))
)

There appear to be up to 6 alternate alleles for each variant.

In [ ]:
np.unique(callset['calldata/GT'])

Inferred a different way using the `GenotypeArray` object ...

In [ ]:
gt = allel.GenotypeArray(callset['calldata/GT'])
gt

In [ ]:
unq_gts = np.unique(gt.to_gt())
unq_gts

In [ ]:
len(unq_gts)

The `GenotypeArray` also tells us that there are no missing calls

In [ ]:
gt.count_missing()

Variants are a combination of SNPs, InDels, and Copy Number variants.

In [ ]:
np.unique(callset['variants/ALT'])

## The Modeling Problem

### Feature selection
For this modeling exercise, we are going to use the variant "ID" - combination of the chromosome position, the "reference" allele, and the "alternative" allele as features for K-Means clustering.  To simplify things, lets just focus on SNPs that are bi-allelic variants.  To do this, we filter for entries where there is only 1 nucleotide in the REF and ALT lists.

There are 8090 single nucleotide reference alleles

In [ ]:
REF = callset['variants/REF']
REF

In [ ]:
is_1bp_ref = np.array(list(map(lambda r: len(r) == 1, REF)), dtype=bool)
sum(is_1bp_ref)

There are 8147 single nucleotide alternative alleles

In [ ]:
# the default import considers only 3 allele alternatives
ALT = callset['variants/ALT']
ALT

In [ ]:
is_1bp_alt = np.array(list(map(lambda a: len(a[0]) == 1 and not any(a[1:]), ALT)), dtype=bool)
sum(is_1bp_alt)

The intersection of the above yields 7946 SNP variants

In [ ]:
is_snp_var = is_1bp_ref & is_1bp_alt
sum(is_snp_var)

We can reduce the feature set further by only considering variants with allele frequencies >= 30

In [ ]:
# get a count of alleles for each variant
ac = gt.count_alleles()
ac

In [ ]:
ac[ac[:,1].argsort()[::-1]][:10, 1]

In [ ]:
# since we're only looking for only bi-allelic SNPs, we're only concerned with the first alternative allele
is_hifreq_snp_var = is_snp_var & (ac[:, 1] >= 30)
sum(is_hifreq_snp_var)

We are now down to ~1700 features, which is certainly more manageable than the ~8300 we started with

### Data transformation

Machine learning algorithms work best on numerical data.

We can convert the genotypes into integer values easily using bit-packing provided by the `GenotypeArray.to_packed` method

In [ ]:
xx = allel.GenotypeArray([
    [[0,0], [0,1], [1,0], [1,1]]
])
xx

In [ ]:
xx.to_packed()

From the above we see that a values of 1 and 16 are effectively equivalent - they correspond to the same genotype: heterozygous for the alt allele.  Where 17 is homozygous for the alt allele.  We can recode these to:
* 16 --> 1
* 17 --> 2

We'll apply this transformation to our GenoTypeArray, which we'll use as our data for training, and apply the filter for only high frequency SNPs that we generated above.  We also want the data entries to list samples along the rows and variants along the columns, so we'll use the transpose of the coded GenoTypeArray.

In [ ]:
gt_coded = gt.to_packed()[is_hifreq_snp_var,:]
gt_coded[gt_coded == 16] = 1
gt_coded[gt_coded == 17] = 2
gt_coded.transpose()

Use `CHROM`, `POS`, `REF`, and `ALT` to create variant feature IDs

In [ ]:
features = [
    '{}-{}-{}-{}'.format(c, p, r, a[0])
    for c, p, r, a in zip(
        callset['variants/CHROM'], 
        callset['variants/POS'], 
        callset['variants/REF'], 
        callset['variants/ALT']
    )]
features = np.array(features)[is_hifreq_snp_var]
features

Let's read in the panel metadata to get class labels - the geographic location that each sample originated from.  To simplify the clustering, we'll only focus on the following populations:

* GBR: British from England and Scotland
* ASW: African Ancestry in Southwest US
* CHB: Han Chinese in Bejing, China

In [ ]:
import pandas as pd

classes = pd.read_table('integrated_call_samples_v3.20130502.ALL.panel', usecols=['sample', 'pop'])
classes = classes[classes['pop'].isin(['GBR', 'ASW', 'CHB'])].copy()
classes.head()

In [ ]:
%matplotlib inline
classes.groupby('pop').count().reset_index().plot.bar('pop')

The distribution of samples across these three populations looks reasonable.


Let's now create the data frame to feed into model training

In [ ]:
data = classes.merge(
    pd.concat((
        pd.Series(callset['samples'], name='sample'),
        pd.DataFrame(gt_coded.transpose(), columns=features)),
        axis=1),
    on='sample'
)
data.sample(10)

Ensure 3 populations to cluster

In [ ]:
data['pop'].unique()

In [ ]:
data.shape

## Training
This dataset is small, only 255 observations, but should give an idea of how to use the built-in KMeans algorithm.

Let's use an 80/20 ratio for the train/test split.

In [ ]:
from math import ceil, floor

train_data = data.sample(frac=.8, random_state=1024)
test_data = data[~data['sample'].isin(train_data['sample'])].copy()

print('Observations')
print(f'training: {train_data.shape[0]}')
print(f'test: {test_data.shape[0]}')

In [ ]:
train_labels = train_data[['sample', 'pop']].copy().set_index('sample')
train_labels['pop'] = pd.Categorical(train_labels['pop'])
train_data = train_data.copy().drop(columns='pop').set_index('sample')

test_labels = test_data[['sample', 'pop']].copy().set_index('sample')
test_labels['pop'] = pd.Categorical(test_labels['pop'])
test_data = test_data.copy().drop(columns='pop').set_index('sample')

In [ ]:
from sagemaker import KMeans, get_execution_role

In [ ]:
role = get_execution_role()
bucket = 'pwyming-sagemaker-us-west-2'

In [ ]:
data_location = 's3://{}/sagemaker/genome-kmeans/data'.format(bucket)
output_location = 's3://{}/sagemaker/genome-kmeans/output'.format(bucket)

print('training data will be uploaded to: {}'.format(data_location))
print('training artifacts will be uploaded to: {}'.format(output_location))

kmeans = KMeans(role=role,
                train_instance_count=2,
                train_instance_type='ml.c4.8xlarge',
                output_path=output_location,
                k=3,
                data_location=data_location)

In [ ]:
%%time

kmeans.fit(kmeans.record_set(np.float32(train_data.values)))

## Inference Endpoint Deployment

In [ ]:
%%time

kmeans_predictor = kmeans.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

In [ ]:
%%time

result = kmeans_predictor.predict(np.float32(train_data))
clusters = np.int0([r.label['closest_cluster'].float32_tensor.values[0] for r in result])

Let's see how these predicted clusters map to the real classes

First, how well did the training set cluster?

In [ ]:
pd.crosstab(train_labels['pop'], columns=clusters, colnames=['cluster'])

From this cross tabulation:
* Cluster 0 primarily identifies CHB
* Cluster 1 primarily identifies GBR
* Cluster 2 exclusively identifies ASW

The clustering results are roughly the same on the test data

In [ ]:
result = kmeans_predictor.predict(np.float32(test_data))
clusters = np.int0([r.label['closest_cluster'].float32_tensor.values[0] for r in result])
pd.crosstab(test_labels['pop'], columns=clusters, colnames=['cluster'])

## Bottom Line

The mixture of populations in the clusters may be interpretted as individuals with mixed ancestry.  Also, the clustering could be improved further if there was additional dimensionality reduction (e.g. via PCA), more samples, or both.  

### (Optional) Delete the Endpoint
If you're ready to be done with this notebook, make sure run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
print(kmeans_predictor.endpoint)

In [ ]:
import sagemaker
sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)